# Payment Matching
At Shopee, bank transfer is a payment method in most countries. When a buyer chooses to place an order using bank transfer, he/she is supposed to make the transfer within 2 days after he/she places the order.

After he/she makes the transfer, Shopee will receive a bank statement from the bank and Shopee needs to compare and match the bank statement with the checkout information in order to confirm that this particular order has been paid. This process is called payment matching.

Two criteria need to be met in order to match a bank statement with a checkout:
- Amount match: Statement amount equals checkout amount.
- Name match: Statement description “matches” checkout buyer name (Note: statement description usually contains buyer name)

A proper match occurs when both the amount and the name matches on both bank statement and checkout list.

# Preparation of the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install fuzzywuzzy

import warnings
warnings.filterwarnings('ignore')

import re
from fuzzywuzzy import fuzz

In [2]:
bank = pd.read_csv('/content/drive/MyDrive/Datasets/NDSC/bank_statement.csv')
check = pd.read_csv('/content/drive/MyDrive/Datasets/NDSC/checkout.csv')

In [3]:
bank.head()

,stmt_id,stmt_amount,desc
0,1,100.00,INSTANT TRANSFER BAW CGANBAWI YIWHI * LEZC ...
1,2,23.77,WOOG CARIZA NIWSI YO
2,3,18.00,ZOMAZAW MABIX ATMA * BUGW * KZBUGW - TRANSFER...
3,4,21.87,TRANSFER GO LA/FA AHHA PUYIMA * YEBF ...
4,5,53.68,INSTANT TRANSFER FABZUN WIQAT JIW FA* TEGF ...


In [4]:
print(f"This dataset contains {bank.shape[0]} rows and {bank.shape[1]} columns.")

This dataset contains 240000 rows and 3 columns.


In [5]:
check.head()

,ckt_id,ckt_amount,buyer_name
0,1,20.00,SAVMUEJICE EPHOCP
1,2,14.91,LUHALLAK ZGANFAP HAIQAW VIP LOHK. CIQTI
2,3,13.20,SUQ UNAISIE
3,4,36.90,GOZK BIHKAUT LIV LATAPI
4,7,26.29,HUY ZABIRAN HATIZAN WIHMI IRYIZ


In [6]:
print(f"This dataset contains {check.shape[0]} rows and {check.shape[1]} columns.")

This dataset contains 240000 rows and 3 columns.


# Removing Unwanted Characters
Removing unwanted characters and 'transfer' word on bank_statement.

In [7]:
bank['desc']        = bank['desc'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
check['buyer_name'] = check['buyer_name'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
bank['desc']        = bank['desc'].apply(lambda x: re.sub(r"TRANSFER", ' ', x))
check['buyer_name'] = check['buyer_name'].apply(lambda x: re.sub(r"TRANSFER", ' ', x))

In [8]:
bank.head()

,stmt_id,stmt_amount,desc
0,1,100.00,INSTANT BAW CGANBAWI YIWHI LEZC HXA ZLLEZC F...
1,2,23.77,WOOG CARIZA NIWSI YO
2,3,18.00,ZOMAZAW MABIX ATMA BUGW KZBUGW TO A C
3,4,21.87,GO LA FA AHHA PUYIMA YEBF GNA BFUNDSYYEBF GNA
4,5,53.68,INSTANT FABZUN WIQAT JIW FA TEGF PYA ATTEGF ...


In [9]:
check.head()

,ckt_id,ckt_amount,buyer_name
0,1,20.00,SAVMUEJICE EPHOCP
1,2,14.91,LUHALLAK ZGANFAP HAIQAW VIP LOHK CIQTI
2,3,13.20,SUQ UNAISIE
3,4,36.90,GOZK BIHKAUT LIV LATAPI
4,7,26.29,HUY ZABIRAN HATIZAN WIHMI IRYIZ


In [10]:
# Sorting Values for efficiency and convert the strings to sets for better data structure and better time complexity
bank = bank.sort_values(by=['stmt_amount'])
check = check.sort_values(by=['ckt_amount'])

# Converting the 'description' and 'buyer name' to SET Data Type so there are not duplicate words.
bank['desc'] = bank['desc'].apply(lambda x: set(x.split()))
check['buyer_name'] = check['buyer_name'].apply(lambda x: set(x.split()))

In [11]:
bank.head()

,stmt_id,stmt_amount,desc
37740,37741,0.01,"{ZIFY, ZAUGAH, BUDABBSHOPEEAW}"
150536,150537,0.01,"{AI, FUNDSTYA, A, TO, PIHK, GEPC, PGGEPC, C, T..."
226724,226725,0.01,"{WOK, PAQB, PEE}"
230963,230964,0.01,"{FAWXANAH, GOFONR, VTVUQINR, WAQIN, VUQINR, NOL}"
115857,115858,0.01,"{A, TO, CIBKI, ACP, C, SHOPEE, CLA, BULVIBJ, I..."


In [12]:
check.head()

,ckt_id,ckt_amount,buyer_name
77722,106443,0.01,"{CIN, RAXVAN, WXAKIH}"
69694,95499,0.01,"{ZIDGI, DUX, RAMAXADI, RUWVICWI}"
35591,48817,0.01,"{ACP, IBAXILAY, LAYGAB}"
133597,183132,0.01,"{BUDABBAW, ZAUGAH, XIH, AXIWIH, ZIFYI}"
167547,229799,0.01,"{AI, PIHK, PAHK}"


# 1) Match the Price and Description
1. Go through the unique prices one by one
2. For each price, query all the bank statements and transactions
3. For each transaction, find a bank statement that matches best.
4. If the transaction cannot find a fitting bank statement, it will be stored to be processed later using fuzzy search.

In [15]:
# Sorted unique price

prices = sorted(list(set(check['ckt_amount'])))

In [32]:
# Match the price & description

bs = bank.values.tolist()
co = check.values.tolist()

curr_bank, curr_trans, pend_bank, pend_trans = [], [], [], []
answer = []
no_ans_trans, no_ans_bank = [], []

for price in prices:
    while bs and bs[0][1] == price:
        curr_bank.append(bs.pop(0))
    while co and co[0][1] == price:
        curr_trans.append(co.pop(0))

    for trans in curr_trans:
        found = False
        curname = trans[2]
        bk = list(filter(lambda x: x[2].intersection(curname), curr_bank))
        if bk:
            b = max(bk, key= lambda x: len(x[2].intersection(curname)))
            answer.append((trans, b))
            curr_bank.remove(b)
        else:
            no_ans_trans.append(trans)
            
    no_ans_bank.extend(curr_bank)
            
    curr_bank, curr_trans = [], []

In [54]:
# Preview the Transactions
pd.DataFrame(answer)

,0,1
0,"[106443, 0.01, {CIN, RAXVAN, WXAKIH}]","[114360, 0.01, {BTA, CIN, GRREGZ, REGZ, FUNDS,..."
1,"[95499, 0.01, {ZIDGI, DUX, RAMAXADI, RUWVICWI}]","[201905, 0.01, {DUX, ATOGQ, ZIDGI, RAMAXADI, I..."
2,"[48817, 0.01, {ACP, IBAXILAY, LAYGAB}]","[115858, 0.01, {A, TO, CIBKI, ACP, C, SHOPEE, ..."
3,"[183132, 0.01, {BUDABBAW, ZAUGAH, XIH, AXIWIH,...","[37741, 0.01, {ZIFY, ZAUGAH, BUDABBSHOPEEAW}]"
4,"[229799, 0.01, {AI, PIHK, PAHK}]","[150537, 0.01, {AI, FUNDSTYA, A, TO, PIHK, GEP..."
...,...,...
235920,"[171945, 200.0, {LIJASAWI, POH}]","[136552, 200.0, {IMLO, ATZOGEH, LIJASAWI, ZOGE..."
235921,"[179059, 200.0, {JQ, TUYIAV, GAKJAZI}]","[138826, 200.0, {TUYIAV, NIQSVEC, XKTUYIAV, JQ..."
235922,"[81122, 200.0, {NKULAGA, QUJUX}]","[55752, 200.0, {SDVEW, DDQW, A, TO, C, QUJUX, ..."
235923,"[137964, 200.0, {HADIEAG, SUWASSAH}]","[192222, 200.0, {RAFE, NID, V, DEYYOF, HADIEAG..."


In [36]:
# Number of transaction that does not match
print(len(no_ans_bank), len(no_ans_trans))

4075 4075


In [37]:
no_ans_bank[:5]

[[134561, 0.1, {'AYRIHAW', 'NIMPI', 'SRZB', 'YUQOM', 'ZB'}],
 [122394, 0.1, {'A', 'C', 'GQ', 'PUJPANIFA', 'SHOPEE', 'TO', 'WAJFAPA'}],
 [158581, 0.1, {'A', 'AD', 'ALAZJAJHICA', 'DILGI', 'GO', 'NA', 'ZA'}],
 [9325,
  0.11,
  {'A', 'B', 'BAIC', 'BJTROB', 'C', 'PALA', 'TO', 'TROB', 'V', 'ZAYTRIJI'}],
 [145415, 0.11, {'A', 'DA', 'JA', 'KULBAGIFAB', 'VO'}]]

In [38]:
no_ans_trans[:5]

[[309643, 0.1, {'ALA', 'ZJAJHICA'}],
 [275938, 0.1, {'PANIFA', 'PUJ', 'WAJFAPPA'}],
 [117315, 0.1, {'AYRIHAWYUQOMB'}],
 [141553, 0.11, {'BAGIFAB'}],
 [8350, 0.11, {'PALAZAYTRJI'}]]

# 2) Transactions Which Not Matched (yet)
For transaction that does not match, using FuzzyWuzzy.

In [39]:
# pick the 'id' only

no_id_bank, no_id_trans = [], []
for x in range(len(no_ans_bank)):
    no_id_bank.append(no_ans_bank[x][0])
    no_id_trans.append(no_ans_trans[x][0])

In [42]:
# get the subset of the DataFrame that previously has no match

check_new = check[check['ckt_id'].isin(no_id_trans)]
bank_new = bank[bank['stmt_id'].isin(no_id_bank)]

In [43]:
# bank DataFrame which has no match transaction
bank_new.head()

,stmt_id,stmt_amount,desc
134560,134561,0.10,"{ZB, YUQOM, NIMPI, SRZB, AYRIHAW}"
122393,122394,0.10,"{WAJFAPA, A, TO, GQ, C, SHOPEE, PUJPANIFA}"
158580,158581,0.10,"{AD, A, ZA, DILGI, GO, NA, ALAZJAJHICA}"
9324,9325,0.11,"{BAIC, A, TO, TROB, V, C, B, BJTROB, ZAYTRIJI,..."
145414,145415,0.11,"{VO, A, KULBAGIFAB, DA, JA}"


In [45]:
# checkout DataFrame which has no match transaction
check_new.head()

,ckt_id,ckt_amount,buyer_name
225903,309643,0.10,"{ALA, ZJAJHICA}"
201280,275938,0.10,"{PANIFA, PUJ, WAJFAPPA}"
85648,117315,0.10,{AYRIHAWYUQOMB}
103356,141553,0.11,{BAGIFAB}
6114,8350,0.11,{PALAZAYTRJI}


In [46]:
# Convert to string

bank_new['desc'] = bank_new['desc'].apply(lambda x: ' '.join(sorted(list(x))))
check_new['buyer_name'] = check_new['buyer_name'].apply(lambda x: ' '.join(sorted(list(x))))

In [47]:
check_new.head()

,ckt_id,ckt_amount,buyer_name
225903,309643,0.10,ALA ZJAJHICA
201280,275938,0.10,PANIFA PUJ WAJFAPPA
85648,117315,0.10,AYRIHAWYUQOMB
103356,141553,0.11,BAGIFAB
6114,8350,0.11,PALAZAYTRJI


In [48]:
# unique prices for unmatch transaction

prices_new = sorted(list(set(check_new['ckt_amount'])))

In [49]:
bs2, co2 = bank_new.values.tolist(), check_new.values.tolist()

curr_bank2, curr_trans2 = [], []
answer2 = []

for price in prices_new:
    while bs2 and bs2[0][1] == price:
        curr_bank2.append(bs2.pop(0))
    while co2 and co2[0][1] == price:
        curr_trans2.append(co2.pop(0))
    for trans in curr_trans2:
        curname = trans[2]
        b = max(curr_bank2, key= lambda x: fuzz.partial_ratio(curname, x[2]))
        answer2.append((trans, b))
        curr_bank2.remove(b)
    curr_bank2, curr_trans2 = [], []

In [53]:
# Preview the Transactions
pd.DataFrame(answer2)

,0,1
0,"[309643, 0.1, ALA ZJAJHICA]","[158581, 0.1, A AD ALAZJAJHICA DILGI GO NA ZA]"
1,"[275938, 0.1, PANIFA PUJ WAJFAPPA]","[122394, 0.1, A C GQ PUJPANIFA SHOPEE TO WAJFAPA]"
2,"[117315, 0.1, AYRIHAWYUQOMB]","[134561, 0.1, AYRIHAW NIMPI SRZB YUQOM ZB]"
3,"[141553, 0.11, BAGIFAB]","[145415, 0.11, A DA JA KULBAGIFAB VO]"
4,"[8350, 0.11, PALAZAYTRJI]","[9325, 0.11, A B BAIC BJTROB C PALA TO TROB V ..."
...,...,...
4070,"[251960, 198.86, ZAQYIEQAWUGLAQZACI]","[207840, 198.86, A C CUQ KYOGFZK LAQ OGFZK TO ..."
4071,"[261405, 200.0, NKOCWSAQQEE]","[210249, 200.0, ASSEXH BFA FUNDSBFA INSTANT NK..."
4072,"[304559, 200.0, EHAVAWCUKI]","[31156, 200.0, A C EHA NWOZE TO VAWCUKI ZJNWOZE]"
4073,"[153115, 200.0, JULNTAFGIJA]","[11798, 200.0, A GAKATNIA JUL L NFOXXEE NTAFGI..."


In [51]:
# Construct final dataframe
fs1 = [(x[0][0], x[1][0]) for x in answer]
fs2 = [(x[0][0], x[1][0]) for x in answer2]

fs = pd.DataFrame(fs1+fs2)
fs.columns=['ckt_id', 'stmt_id']
fs=fs[['stmt_id', 'ckt_id']]

In [56]:
fs

,stmt_id,ckt_id
0,114360,106443
1,201905,95499
2,115858,48817
3,37741,183132
4,150537,229799
...,...,...
239995,207840,251960
239996,210249,261405
239997,31156,304559
239998,11798,153115


# Output the CSV

In [58]:
# Output CSV
print(fs.shape)
fs.to_csv('submission.csv', index=False)

(240000, 2)
